In [7]:
import backtrader as bt
import pickle
import backtrader.indicators as btind
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
from datetime import datetime, timedelta
from sklearn import svm
import pandas as pd
import math
from sklearn import preprocessing
from math import sqrt
from technicalSignals import momentum,SMA,inBBands
from __future__ import division



In [8]:
tickers=['AAPL','AMZN','PEP','GOOGL','MSFT','FB','INTC','CSCO','CMCSA','NVDA','NFLX','BKNG','ADBE','AMGN','TXN','AVGO','PYPL','GILD','COST','QCOM']       

meanprice=list()
for ticker in tickers:
    price=pd.read_csv('/home/andrea/Desktop/NLFF/TechnicalDatasetUpdated/'+ticker+'.csv')
    meanprice.append(price['open'].iloc[math.floor(len(price)*0.9)])
print(meanprice)

[187.31, 1581.57, 100.0, 1080.44, 97.4647, 183.88, 54.49, 43.84, 32.57, 241.99, 332.93, 2130.65, 239.21, 178.69, 111.27, 239.442, 80.34, 68.21, 199.1278, 57.42]


In [9]:
# ===========================================    Trading Library utilities

    
# class Sizer(bt.Sizer):
#     params = dict(stake=1)

#     def _getsizing(self, comminfo, cash, data, isbuy):
#         dt, i = self.strategy.datetime.date(), data._id
#         s = self.p.stake * (1 + (not isbuy))
#         print('{} Data {} OType {} Sizing to {}'.format(
#             dt, data._name, ('buy' * isbuy) or 'sell', s))

#         return s

    
class Strategy(bt.Strategy):
    params = dict(
        pred_threshold = None,        #buy if prediction more than this threshold (prediction values normalized 1: most secure prediction)
        num_pred_over_threshold = None, #buy if more than this number of prediction over threshold
        forecast_window = None,
        pred_threshold_pos=None,
        pred_threshold_neg=None,
        predictions = None,
        verbose = None,
        tol=None,
        value=None
    )

    def log(self, txt, dt=None):
        dt = dt or self.datetime.datetime().strftime('%Y-%m-%d %H:%M:%S')
        print('%s, %s' % (dt, txt))

    def notify_order(self, order):
        if self.verbose:
            if order.status in [order.Submitted, order.Accepted]:
                return
            if order.status in [order.Completed]:
                if order.isbuy():
                    self.log('BUY EXECUTED, %.2f' % order.executed.price)
                elif order.issell():
                    self.log('SELL EXECUTED, %.2f' % order.executed.price)
            elif order.status in [order.Canceled, order.Margin, order.Rejected]:
                self.log('Order Canceled/Margin/Rejected')

        
    def __init__(self):
        self.planned_sell = {}  # PLANNED SELL FOR EACH BUY AFTER TREND WINDOW dict: selldate, stock -> amount
        self.forecast_window = self.params.forecast_window
        self.predictions = self.params.predictions
        self.pred_threshold=self.params.pred_threshold
        
        self.num_pred_over_threshold = self.params.num_pred_over_threshold
        self.verbose = self.params.verbose
        self.tol = self.params.tol
        self.value=self.params.value
        

    def next(self): 
        positions = {}
        for i, d in enumerate(self.datas):
            #dt = self.datetime.date()
            dt = self.datetime.datetime().strftime('%Y-%m-%d %H:%M:%S')
            
            if(len((np.where(self.predictions[i]['Unnamed: 0']==dt)[0]))>0):
                time_idx = np.where(self.predictions[i]['Unnamed: 0']==dt)[0][0]
                dn = d._name
                ticker_pred = self.predictions[i][d._name][time_idx]

                previous_pred = self.predictions[i][d._name].iloc[time_idx-self.num_pred_over_threshold:time_idx]

                previous_pred_pos = sum([1 if p>0.5+self.pred_threshold else 0 for p in previous_pred]) #count prev over threshold
                previous_pred_neg = sum([1 if p<0.5-self.pred_threshold else 0 for p in previous_pred])
                pos = self.getposition(d).size
                positions[dn]=pos
                # Strategy: buy if abs(pred)> threshold or if more than n predictions positive
                quantity=math.ceil(self.value/(2800*meanprice[i]))*10
                
                
                if(ticker_pred>self.pred_threshold+0.5 or ticker_pred<0.5-self.pred_threshold):
                    # Buy and set planned sell if prediction positive
                    if(ticker_pred > 0.5 and previous_pred_pos>=self.num_pred_over_threshold*self.tol):
                        self.buy(data = d, size=quantity)                
                        idx_to_sell = time_idx+self.forecast_window
                        self.planned_sell[(dn,idx_to_sell)] = quantity
                    elif(ticker_pred<0.5 and previous_pred_neg>=self.num_pred_over_threshold*self.tol):
                        self.sell(data = d, size=quantity)                
                        idx_to_sell = time_idx+self.forecast_window
                        self.planned_sell[(dn,idx_to_sell)] = -quantity
                #Execute sell planned for now
                if (dn,time_idx) in self.planned_sell:
                    amount = self.planned_sell[(dn,time_idx)]
                    if(amount>0):
                        self.sell(data = d, size=amount)
                    else:
                        self.buy(data = d, size=amount)
                    del self.planned_sell[(dn,time_idx)] 
            
        if self.verbose:
                print('{} Positions {}'.format(dt, positions))

                
                
                
def printTradeAnalysis(analyzer):
    total_open = analyzer.total.open
    total_closed = analyzer.total.closed
    total_won = analyzer.won.total
    total_lost = analyzer.lost.total
    win_streak = analyzer.streak.won.longest
    lose_streak = analyzer.streak.lost.longest
    pnl_net = round(analyzer.pnl.net.total,2)
    strike_rate = (total_won / total_closed) * 100
    h1 = ['Total Open', 'Total Closed', 'Total Won', 'Total Lost']
    h2 = ['Strike Rate','Win Streak', 'Losing Streak', 'PnL Net']
    r1 = [total_open, total_closed,total_won,total_lost]
    r2 = [strike_rate, win_streak, lose_streak, pnl_net]
    if len(h1) > len(h2):
        header_length = len(h1)
    else:
        header_length = len(h2)
    print_list = [h1,r1,h2,r2]
    row_format ="{:<15}" * (header_length + 1)
    print("Trade Analysis Results:")
    for row in print_list:
        print(row_format.format('',*row))

def printSQN(analyzer):
    sqn = round(analyzer.sqn,2)
    print('SQN: {}'.format(sqn))
def get_hist_broker(strategy):
    i=1
    portfolioValue = []
    cash = []
    while not np.isnan(strategy.stats.broker.value.array[i]):
        portfolioValue.append(strategy.stats.broker.value.array[i])
        cash.append(strategy.stats.broker.cash.array[i])
        i += 1
        
    return (np.asarray(portfolioValue),np.asarray(cash))    
def annualizedReturn(Return):
    annualized=((1+Return)**(365/59))-1
    return annualized
def compute_SharpeRatio(strategy):
    (portfolioValue, cash) = get_hist_broker(strategy)
    returns = np.asarray(list(strategy.analyzers.returns.get_analysis().values()))
    print(returns)
    sharpe = (np.mean(returns)-0.0003938)/np.std(returns)
    return sharpe


In [10]:
import matplotlib.pylab as pylab
pylab.rcParams['figure.figsize'] = 30, 20  # that's default image size for this interactive session
pylab.rcParams['font.family'] = 'sans-serif'
pylab.rcParams['font.sans-serif'] = ['Bitstream Vera Sans']
pylab.rcParams['font.serif'] = ['Bitstream Vera Sans']
pylab.rcParams["font.size"] = "30"

In [19]:
def simulation(treshold,n_predict_over,init_val,tickers,tollerance,source):
    pred_threshold =treshold        #buy if prediction more than this threshold (prediction values normalized 1: most secure prediction)
    num_pred_over_threshold = n_predict_over  #buy if more than this number of prediction over threshold
    init_value = init_val



    cerebro = bt.Cerebro(stdstats=False)
    cerebro.addobservermulti(bt.observers.BuySell)
    #cerebro.addobservermulti(bt.observers.BuySell)
    #cerebro.addobserver(bt.observers.CashValue)
    cerebro.addobserver(bt.observers.DrawDown)
    cerebro.addobserver(bt.observers.Broker)
    cerebro.broker.setcash(init_value)
    cerebro.broker.setcommission(commission=0.0001)
    predictions=list() 

    evenPlot = True
    for ticker in tickers:
        temppred=pd.read_csv('/home/andrea/Desktop/NLFF/technicalAnalysis/NN/Predictions/Newsetting/newwithadam/'+source+'/'+ticker+'.csv')
        predictions.append(temppred)
        #plt.hist(temppred[ticker], bins = np.arange(0.0, 1.0, 0.01))
        #plt.show()
        dates_test = [datetime.strptime(row, '%Y-%m-%d %H:%M:%S') for row in temppred['Unnamed: 0']]
        
        data = bt.feeds.GenericCSVData(
                    dataname='/home/andrea/Desktop/NLFF/TechnicalDatasetUpdated/GMT+8/'+ticker+'.csv',
                    name=ticker,
                    value=init_value,
                    timeframe = bt.TimeFrame.Minutes, 
                    compression = 15,
                    datetime=1,open=2,high=3,low=4,close=5,volume=6,openinterest=-1,
                    fromdate=dates_test[0],
                    todate=dates_test[-1]+timedelta(minutes=1),
                    reverse=False)
    #         if(evenPlot):
    #             data0 = data
    #             evenPlot = False
    #         else:
    #             data.plotinfo.plotmaster = data0
    #             evenPlot = True
        cerebro.adddata(data)

    #cerebro.addsizer(Sizer)    
     #Up to now only future
    cerebro.addstrategy(Strategy,
                        pred_threshold = pred_threshold,
                        num_pred_over_threshold = num_pred_over_threshold,
                        forecast_window=140,
                        predictions = predictions,
                        verbose = False,
                        tol=tollerance,
                        value=init_value)
    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name="ta")
    cerebro.addanalyzer(bt.analyzers.SQN, _name="sqn")
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharp',timeframe=bt.TimeFrame.Minutes, compression=15,riskfreerate=0.0000028,convertrate=False)
    cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown') 
    cerebro.addanalyzer(bt.analyzers.TimeReturn, _name='returns')

    #print('Starting Portfolio Value: %.2f' % init_value)
    strategy = cerebro.run()[0]
    final_value = cerebro.broker.getvalue()
    #cerebro.plot()
    printTradeAnalysis(strategy.analyzers.ta.get_analysis())
    
    print('Sharp ratio: ',strategy.analyzers.sharp.get_analysis())
    print('Max drwadown: ',strategy.analyzers.drawdown.get_analysis())
    printSQN(strategy.analyzers.sqn.get_analysis())
    print('Final Portfolio Value: %.2f \nGain: %.2f' % (final_value, (100*(final_value-init_value))/init_value))
    print('Annualized Return:',annualizedReturn((final_value-init_value)/init_value))
    return((100*(final_value-init_value))/init_value)

In [20]:
simulation(0.4,0,1000000,tickers,1,'price')

Trade Analysis Results:
               Total Open     Total Closed   Total Won      Total Lost     
               18             23             8              8              
               Strike Rate    Win Streak     Losing Streak  PnL Net        
               34.782608695652172              2              26855.09       
Sharp ratio:  OrderedDict([('sharperatio', -0.03394645021244051)])
Max drwadown:  AutoOrderedDict([('len', 96), ('drawdown', 12.535592991757198), ('moneydown', 129528.7870128015), ('max', AutoOrderedDict([('len', 114), ('drawdown', 12.931796872040064), ('moneydown', 133622.7144445991)]))])
SQN: 0.96
Final Portfolio Value: 903759.28 
Gain: -9.62
Annualized Return: -0.46528412367361316


-9.624071601389954

In [ ]:
#PRICE
gainlist=list()
for over in [0,4,8,12,16,20]:
    gain=list()
    for t in np.arange(0,0.5,0.02):
        #print('Threshold: ',t)
        gain.append(simulation(t,over,1000000,tickers,1,'price'))
    print('Over: '+str(over)+', t: '+str(t))
    plt.plot(np.arange(0.5,1,0.02),gain)
    plt.show()
    gainlist.append(gain)
print('FINAL COMPARISON')
for el in range(0,len(gainlist)):
    plt.plot(np.arange(0.5,1,0.02),gainlist[el])
plt.show()

In [ ]:
print(gainlist[0])

In [ ]:
plt.plot(np.arange(0.5,1,0.02),gain)
plt.show()

In [ ]:
gain=list()
for t in np.arange(0,0.5,2.02):
    print('Threshold: ',t)
    gain.append(simulation(t,0,1000000,tickers,1))
plt.plot(np.arange(0.5,1,0.02),gain)
plt.show()

In [ ]:
gain=list()
for t in np.arange(0,0.5,5.02):
    print('Threshold: ',t)
    gain.append(simulation(t,0,1000000,tickers,1))
plt.plot(np.arange(0.5,1,0.02),gain)
plt.show()

In [ ]:
gain=list()
for t in np.arange(0,0.5,8.02):
    print('Threshold: ',t)
    gain.append(simulation(t,0,1000000,tickers,1))
plt.plot(np.arange(0.5,1,0.02),gain)
plt.show()

In [ ]:
gain=list()
for t in np.arange(0,0.5,12.02):
    print('Threshold: ',t)
    gain.append(simulation(t,0,1000000,tickers,1))
plt.plot(np.arange(0.5,1,0.02),gain)
plt.show()

In [ ]:
#do for one ticke time to make plot
for ticker in tickers:
    

    pred_threshold = 0.3        #buy if prediction more than this threshold (prediction values normalized 1: most secure prediction)
    num_pred_over_threshold = 5  #buy if more than this number of prediction over threshold
    init_value = 100000.0


    cerebro = bt.Cerebro(stdstats=False)
    cerebro.addobservermulti(bt.observers.BuySell)
    cerebro.addobserver(bt.observers.Broker)
    cerebro.addobserver(bt.observers.DrawDown)
    
   
    cerebro.broker.setcash(init_value)
    cerebro.broker.setcommission(commission=0.0001)
    predictions=list() 

    evenPlot = True
    
    temppred=pd.read_csv('/home/andrea/Desktop/NLFF/technicalAnalysis/NN/Predictions/Newsetting/both/'+ticker+'.csv')
    predictions.append(temppred)
    #plt.hist(temppred[ticker], bins = np.arange(0.0, 1.0, 0.01))
    #plt.show()
    dates_test = [datetime.strptime(row, '%Y-%m-%d %H:%M:%S') for row in temppred['Unnamed: 0']]
    data = bt.feeds.GenericCSVData(
                dataname='/home/andrea/Desktop/NLFF/TechnicalDataset/GMT+0/'+ticker+'.csv',
                name=ticker,
                timeframe = bt.TimeFrame.Minutes, 
                compression = 15,
                datetime=1,open=2,high=3,low=4,close=5,volume=6,openinterest=-1,
                fromdate=dates_test[0],
                todate=dates_test[-1]+timedelta(minutes=1),
                reverse=False)
#         if(evenPlot):
#             data0 = data
#             evenPlot = False
#         else:
#             data.plotinfo.plotmaster = data0
#             evenPlot = True
    cerebro.adddata(data)

    #cerebro.addsizer(Sizer)    
     #Up to now only future
    cerebro.addstrategy(Strategy,
                        pred_threshold = pred_threshold,
                        num_pred_over_threshold = num_pred_over_threshold,
                        forecast_window=140,
                        predictions = predictions,
                        verbose = True)
    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name="ta")
    cerebro.addanalyzer(bt.analyzers.SQN, _name="sqn")
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe',timeframe=bt.TimeFrame.Minutes, compression=15) 
    print('Starting Portfolio Value: %.2f' % init_value)
    strategy = cerebro.run()[0]
    final_value = cerebro.broker.getvalue()
    cerebro.plot()
    
    printTradeAnalysis(strategy.analyzers.ta.get_analysis())
    print(strategy.analyzers.sharpe.get_analysis())
    printSQN(strategy.analyzers.sqn.get_analysis())
    print('Final Portfolio Value: %.2f \nGain: %.2f' % (final_value, (100*(final_value-init_value))/init_value))

In [ ]:
## MARKET PERFORMANCE:
class BuyAndHold(bt.Strategy):
    def log(self, txt, dt=None):
        dt = dt or self.datetime.datetime().strftime('%Y-%m-%d %H:%M:%S')
        print('%s, %s' % (dt, txt))

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, %.2f' % order.executed.price)
            elif order.issell():
                self.log('SELL EXECUTED, %.2f' % order.executed.price)
      
    def __init__(self):
        return

    def next(self): 
        for i, d in enumerate(self.datas):
            self.buy(data = d, size=1)                


init_value = 3670.0 #Correct value to buy one stock a the beginning
init_value = 100000

for (init, finish) in TREND_WINDOWs:
    print('\n====================  trend: ',init,' ',finish, ' ==================== \n')
    cerebro = bt.Cerebro()
    cerebro.broker.setcash(init_value)
    cerebro.broker.setcommission(commission=0.0001)
    predictions = pd.read_csv('testPredictions/AllTickers_'+str(init)+'_'+str(finish)+'.csv', index_col = 0)
    dates_test = [datetime.strptime(row, '%Y-%m-%d %H:%M:%S') for row in predictions.index]
    for ticker in tickers:
        data = bt.feeds.GenericCSVData(
                    dataname='/home/simone/Desktop/NLFF/indexes/indexes'+ticker+'.csv',
                    name=ticker,
                    timeframe = bt.TimeFrame.Minutes, 
                    compression = 60,
                    datetime=1,open=2,high=3,low=4,close=5,volume=6,openinterest=-1,
                    fromdate=dates_test[0],
                    todate=dates_test[-1]+timedelta(minutes=1),
                    reverse=False)
        cerebro.adddata(data)
        
    cerebro.addstrategy(BuyAndHold)
    cerebro.addanalyzer(bt.analyzers.SQN, _name="sqn")
    print('Starting Portfolio Value: %.2f' % init_value)
    strategy = cerebro.run()[0]
    final_value = cerebro.broker.getvalue()
    print('Final Portfolio Value: %.2f \nGain: %.2f' % (final_value, final_value/init_value - 1))